# Simulation of DiThR model

## Modules and functions

In [ ]:
#############################
### Modules and functions ###
#############################


#### Modules

using RandomNumbers.MersenneTwisters
using Random
using Distributions
using Plots
using CSV
using DataFrames
using DelimitedFiles
using Glob


### Function ###

## return a random sample from an exponential distribution
function rand_exponential(mean)
    if mean <= 0.0
        error("mean must be positive")
    end
    -mean*log(rand())
end



## Gillespie's algorithm

In [ ]:

@inbounds @fastmath function gillespie_ON_OFF_minimal_retro_nointer(time, IDOX_tot, OFF_0, ON_0, RNA_0, k_incorpo,
                         k_repair, k_on, k_off,k_RNA, k_d, coop_factor)
    compteur=0
    List_time = [0.]
    List_ON = [ON_0]
    List_ON_incorpo = [0]
    List_OFF = [OFF_0]
    List_RNA = [RNA_0]
    k_RNA_cst = k_RNA
    while List_time[end] < time
        rate_vector = [
            k_on * List_OFF[end], k_off * List_ON[end],
            k_RNA * List_ON[end], k_d * List_RNA[end],
            k_incorpo * List_ON[end] * IDOX_tot,
            k_repair * List_ON_incorpo[end]
        ]
        tot_rate = sum(rate_vector)
        t_step = rand_exponential(1 / tot_rate)
        compteur+=1
        push!(List_time,List_time[end] + t_step)
        random_number = rand()
        if random_number <= sum(rate_vector[1:1]) / tot_rate
            #OFF -> ON
            push!(List_ON,List_ON[end] + 1)
            push!(List_OFF,List_OFF[end] - 1)
            push!(List_RNA,List_RNA[end])
            push!(List_ON_incorpo,List_ON_incorpo[end])
        elseif random_number <= sum(rate_vector[1:2]) / tot_rate
            #ON -> OFF
            push!(List_ON,List_ON[end] - 1)
            push!(List_OFF,List_OFF[end] + 1)
            push!(List_RNA,List_RNA[end])
            push!(List_ON_incorpo,List_ON_incorpo[end])
            k_RNA = k_RNA_cst
        elseif random_number <= sum(rate_vector[1:3]) / tot_rate
            #ON -> RNA
            push!(List_ON,List_ON[end])
            push!(List_OFF,List_OFF[end])
            push!(List_RNA,List_RNA[end] + 1)
            push!(List_ON_incorpo,List_ON_incorpo[end])
        elseif random_number <= sum(rate_vector[1:4]) / tot_rate
            #RNA -> ∅
            push!(List_ON,List_ON[end])
            push!(List_OFF,List_OFF[end])
            push!(List_RNA,List_RNA[end] - 1)
            push!(List_ON_incorpo,List_ON_incorpo[end])
        elseif random_number <= sum(rate_vector[1:5]) / tot_rate
            #ON -> ON incorpo
            push!(List_ON,List_ON[end] - 1)
            push!(List_OFF,List_OFF[end])
            push!(List_RNA,List_RNA[end])
            push!(List_ON_incorpo,List_ON_incorpo[end] + 1)
            k_RNA = 0
        elseif random_number <= sum(rate_vector[1:6]) / tot_rate
            # repair reaction ON
            push!(List_ON,List_ON[end] + 1)
            push!(List_OFF,List_OFF[end])
            push!(List_RNA,List_RNA[end])
            push!(List_ON_incorpo,List_ON_incorpo[end] - 1)
            k_RNA = k_RNA_cst * coop_factor
        else
            println("Probability should sum to 1 isn'it?\n")
        end
    end
    return (List_time, List_ON, List_ON_incorpo, List_OFF, List_RNA)
end

## Parameters for the simulation

In [ ]:
### Example parameters
time=400
IDOX_tot= 1
OFF_0=1
ON_0=0
RNA_0=0
k_incorpo=1.442510577
k_repair=1.0911
k_on=1.43
k_off=1.2555
k_RNA=35.4429
k_d=0.16637
coop_factor=1.96

## CSV (traces) files generation

In [ ]:
path =string("../IDOX_project/Model_8bis/review/Model_8bis_no_coop_nospeedup_maxLLE/",IDOX_tot)
mkpath(path)
println(coop_factor)
d_param = DataFrame([[time], [IDOX_tot], [OFF_0], [ON_0], [RNA_0], [k_incorpo], [k_repair], [k_on], [k_off], [k_RNA], [k_d],[coop_factor]], [:time, :IDOX_tot, :OFF_0, :ON_0, :RNA_0, :k_incorpo, :k_repair, :k_on, :k_off, :k_RNA, :k_d, :coop_factor])
file_param = string(path ,"/parameters.csv")
CSV.write(file_param, d_param)
for k in 1:1500
    List_time, List_ON, List_ON_incorpo, List_OFF, List_RNA = gillespie_ON_OFF_minimal_retro_nointer(time, IDOX_tot, OFF_0, ON_0, RNA_0, k_incorpo,k_repair, k_on, k_off,k_RNA, k_d, coop_factor)
    filename = string(path ,"/trace_",k,".csv")
    df = DataFrame([List_time,  List_ON, List_ON_incorpo, List_OFF, List_RNA], [:List_time, :List_ON, :List_ON_incorpo, :List_OFF, :List_RNA])
    CSV.write(filename, df)
end